<a href="https://colab.research.google.com/github/Anshad-Aziz/Indian-Penal-Code-IPC-/blob/main/Indian_Penal_Code_(IPC)_Sections_Information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd

In [2]:
df=pd.read_csv('/content/ipc_sections.csv')

In [3]:
df.head(3)

,Description,Offense,Punishment,Section
0,Description of IPC Section 140\nAccording to s...,Wearing the dress or carrying any token used b...,3 Months or Fine or Both,IPC_140
1,Description of IPC Section 127\nAccording to s...,Receiving property taken by war or depredation...,7 Years + Fine + forfeiture of property,IPC_127
2,Description of IPC Section 128\nAccording to s...,Public servant voluntarily allowing prisoner o...,Imprisonment for Life or 10 Years + Fine,IPC_128


In [4]:
X=df[['Offense','Punishment']].values
y=df['Description'].values

In [5]:
print('Input Features(X):',X[:5])

Input Features(X): [['Wearing the dress or carrying any token used by a soldier, sailor or airman with intent that it may be believed that he is such a soldier, sailor or airman'
  '3 Months or Fine or Both']
 ['Receiving property taken by war or depredation mentioned in sections 125 And 126'
  '7 Years + Fine + forfeiture of property']
 ['Public servant voluntarily allowing prisoner of State or war in his custody to escape'
  'Imprisonment for Life or 10 Years + Fine']
 ['Public servant negligently suffering prisoner of State or war in his custody to escape'
  'Simple Imprisonment 3 Years + Fine']
 ['Aiding escape of, rescuing or harbouring, such prisoner, or offering Any resistance to the recapture of such prisoner'
  'Imprisonment for Life or 10 Years + Fine']]


In [6]:
print('Input Features(X):',y[:5])

Input Features(X): ['Description of IPC Section 140\nAccording to section 140 of Indian penal code, Whoever, not being a soldier, sailor or airman in the Military, Naval or Air service of the Government of India, wears any garb or carries any token resembling any garb or token used by such a soldier, sailor or airman with the intention that it may be believed that he is such a soldier, sailor or airman, shall be punished with imprisonment of either description for a term which may extend to three months, or with fine which may extend to five hundred rupees, or with both.\n\n\nIPC 140 in Simple Words\nIf someone who is not a military member wears a uniform or carries something resembling a military uniform to deceive others into believing they are a soldier, sailor, or airman, they can be punished with up to three months in jail, a fine of up to five hundred rupees, or both.'
 'Description of IPC Section 127\nAccording to section 127 of Indian penal code, Whoever receives any property k

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder

In [8]:
print('Data Types')
print('Offense:',X[:,0].dtype)

Data Types
Offense: object


In [9]:
print('Punishment:',X[:,1].dtype)

Punishment: object


In [10]:
X[:,0]=X[:,0].astype(str)
X[:,1]=X[:,1].astype(str)

In [11]:
X_text=X[:,0]+" "+X[:,1]

In [12]:
vectorizer=CountVectorizer()
X_numerical=vectorizer.fit_transform(X_text)

In [13]:
label_encoder=LabelEncoder()
y_numerical=label_encoder.fit_transform(X_text)

In [14]:
X_train,X_test,y_train,y_test=train_test_split(X_numerical,y_numerical,test_size=0.2,random_state=42)

In [15]:
print("shape of X_numerical:",X_numerical.shape)
print("shape of y_numerical:",y_numerical.shape)

shape of X_numerical: (444, 992)
shape of y_numerical: (444,)


In [16]:
from keras.models import Sequential
from keras.layers import Dense

In [20]:
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(len(np.unique(y_numerical)), activation='softmax'))

In [21]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 64)                63552     
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 384)               12672     
                                                                 
Total params: 78304 (305.88 KB)
Trainable params: 78304 (305.88 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [25]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.preprocessing import LabelEncoder  # Add this import
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [26]:
df=pd.read_csv('/content/ipc_sections.csv')

In [27]:
X=df[['Offense','Punishment']].astype(str).values
y=df['Description'].values

In [28]:
label_encoder=LabelEncoder()
y=label_encoder.fit_transform(y)

In [29]:
output_neurons=len(np.unique(y))

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
X_text_train = X_train[:, 0] + ' ' + X_train[:, 1]
X_text_test = X_test[:, 0] + ' ' + X_test[:, 1]


In [32]:
vectorizer = CountVectorizer()
X_train_numerical = vectorizer.fit_transform(X_text_train)
X_test_numerical = vectorizer.transform(X_text_test)


In [33]:
sample_weights = compute_sample_weight('balanced', y_train)

In [34]:
# Define the neural network model with adjustments
model = Sequential()
model.add(Dense(128, input_dim=X_train_numerical.shape[1], activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

# Set the number of output neurons to match the unique classes
model.add(Dense(output_neurons, activation='softmax'))

# Compile the model with sample weights
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [35]:
history_weighted = model.fit(X_train_numerical.toarray(), y_train, epochs=1000, batch_size=100, validation_split=0.2, sample_weight=sample_weights)

Epoch 1/1000
1/3 [=========>....................] - ETA: 8s - loss: 6.0869 - accuracy: 0.0000e+00

3/3 [==============================] - 5s 156ms/step - loss: 6.0917 - accuracy: 0.0035 - val_loss: 6.0961 - val_accuracy: 0.0000e+00
Epoch 2/1000
1/3 [=========>....................] - ETA: 0s - loss: 6.0696 - accuracy: 0.0000e+00

3/3 [==============================] - 0s 31ms/step - loss: 6.0730 - accuracy: 0.0000e+00 - val_loss: 6.1035 - val_accuracy: 0.0000e+00
Epoch 3/1000
1/3 [=========>....................] - ETA: 0s - loss: 6.0839 - accuracy: 0.0100

3/3 [==============================] - 0s 35ms/step - loss: 6.0658 - accuracy: 0.0035 - val_loss: 6.1125 - val_accuracy: 0.0000e+00
Epoch 4/1000
1/3 [=========>....................] - ETA: 0s - loss: 6.0515 - accuracy: 0.0100

3/3 [==============================] - 0s 34ms/step - loss: 6.0469 - accuracy: 0.0035 - val_loss: 6.1227 - val_accuracy: 0.0000e+00
Epoch 5/1000
1/3 [=========>....................] - ETA: 0s - loss: 6.0139 - accuracy: 0.0300

...

3/3 [==============================] - 0s 34ms/step - loss: 0.8833 - accuracy: 0.7606 - val_loss: 23.2804 - val_accuracy: 0.0000e+00
Epoch 997/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.9397 - accuracy: 0.7400

3/3 [==============================] - 0s 68ms/step - loss: 0.8263 - accuracy: 0.7711 - val_loss: 23.2820 - val_accuracy: 0.0000e+00
Epoch 998/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.7789 - accuracy: 0.7600

3/3 [==============================] - 0s 43ms/step - loss: 0.8292 - accuracy: 0.7676 - val_loss: 23.2885 - val_accuracy: 0.0000e+00
Epoch 999/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.9778 - accuracy: 0.7100

3/3 [==============================] - 0s 44ms/step - loss: 0.8186 - accuracy: 0.7676 - val_loss: 23.3071 - val_accuracy: 0.0000e+00
Epoch 1000/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.9329 - accuracy: 0.7400

3/3 [==============================] - 0s 60ms/step - loss: 0.8850 - accuracy: 0.7535 - val_loss: 23.3372 - val_accuracy: 0.0000e+00


In [36]:
test_loss,test_accuracy=model.evaluate(X_test_numerical.toarray(),y_test)

3/3 [==============================] - 0s 7ms/step - loss: 23.2762 - accuracy: 0.0000e+00


In [37]:
print(f"Test Loss:{test_loss:.4f}")
print(f"Test Accuracy:{test_accuracy:.4f}")

Test Loss:23.2762
Test Accuracy:0.0000


In [38]:
input_text='robbery'

In [39]:
input_numerical=vectorizer.transform([input_text])
prediction=model.predict(input_numerical.toarray())
predicted_label=np.argmax(prediction)
predicted_label_original=label_encoder.inverse_transform([predicted_label])

1/1 [==============================] - 0s 102ms/step


In [40]:
print(predicted_label_original)

['Description of IPC Section 392\nAccording to section 392 of Indian penal code, Whoever commits robbery shall be punished with rigorous imprisonment for a term which may extend to ten years, and shall also be liable to fine; and, if the robbery be committed on the highway between sunset and sunrise, the imprisonment may be extended to fourteen years.\n\n\nIPC 392 in Simple Words\nIf someone commits robbery, they may face rigorous imprisonment for up to ten years and a fine; if it happens on a highway between sunset and sunrise, the imprisonment may be extended to fourteen years.']
